# Notebook for Creating Dynamic Data Sets

### Read Edges and Busses

In [ ]:
import pandas as pd
import numpy as np

# Read Files
busses = pd.read_excel('Busses_Timeseries_Table.xlsx')
edges = pd.read_excel('Edges_Timeseries_Table.xlsx')

### Node Splitting

In [9]:
import random
# Use busses and edges table
def split_node(busses, edges, splitAt):
    # Update Values Node one
    busses.loc[busses['bus_id'] == splitAt, 'mw_norm'] = busses.mw_norm / 2
    busses.loc[busses['bus_id'] == splitAt, 'min_ecomin'] = busses.min_ecomin / 2
    busses.loc[busses['bus_id'] == splitAt, 'max_ecomax'] = busses.max_ecomax / 2
    # h & c stays same (Needs to be solved for again)
    
    # Add Timeseries for New Node
    busAtTimeseries = busses[busses['bus_id'] == splitAt]
    newNode = busses["bus_id"].max() + 1
    busAtTimeseries.loc[:, 'bus_id'] = newNode
    busses = pd.concat([busses, busAtTimeseries])
    busses = busses.sort_values(by=['datetime_beginning_utc', 'bus_id'], ascending=True)
    
    #Create Edge, sample x and limit
    xAverage = edges[edges['X'] != 0]['X'].mean()
    limAverage = edges[edges['Lim MVA A'] != 0]['Lim MVA A'].mean()
    xVal = xAverage * (1+ random.randint(-1, 1)/100)
    limVal = limAverage * (1+ random.randint(-1, 2)/100)
    edge1 = pd.DataFrame({'From Number':[splitAt], 'To Number':[newNode], 'X':[xVal], 'Lim MVA A': [limVal]})
    timestamps = edges['datetime_beginning_utc'].unique()

    # Cross Join to turn grid into timeseries data (same grid)
    timeDF = pd.DataFrame(timestamps, columns=['datetime_beginning_utc'])
    edgeTimeseries = pd.merge(timeDF[['datetime_beginning_utc']], edge1, how='cross')
    edges = pd.concat([edges, edgeTimeseries])
    edges = edges.sort_values(by=['datetime_beginning_utc', 'From Number'], ascending=True)
    
    # Add one year
    busses['datetime_beginning_utc'] += pd.DateOffset(years=1)
    edges['datetime_beginning_utc'] += pd.DateOffset(years=1)
    
    busses.reset_index(inplace = True, drop = True)
    edges.reset_index(inplace = True, drop = True)
    
    return (busses, edges)

In [33]:
# Perform Split
nodeIndex = 0
bussesSplit, edgesSplit = split_node(busses, edges, nodeIndex)
bussesSplit.reset_index(inplace = True, drop = True) # Adjust index
edgesSplit.reset_index(inplace = True, drop = True)

C:\Users\nadin\AppData\Local\Temp\ipykernel_20448\2854311747.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  busAtTimeseries.loc[:, 'bus_id'] = newNode


In [54]:
bussesSplit.to_csv("Busses_Timeseries_Table_Set2023.csv")
edgesSplit.to_csv("Edges_Timeseries_Table_Set2023.csv")

### Node Concatenation

In [270]:
# Short version with group by
def concat_node(busses, edges, concatTo):
    delNode = busses["bus_id"].max()
    
    #Nodes - make node id identical and then aggregate for identical id and time
    busses.loc[busses['bus_id'] == delNode, 'bus_id'] = concatTo    
    wm = lambda x: np.average(x, weights=busses.loc[x.index, "mw_norm"])
    agg_functions = {'load_area': 'first', 'mw_norm': 'sum', 'solTheta': 'sum', 'min_ecomin': 'min', 'max_ecomax': 'sum', 'inter_start_cost': wm, 'incremental_price': wm, 'solGenerate': 'sum', 'solOn': 'max'}
    busses = busses.groupby(['datetime_beginning_utc','bus_id']).aggregate(agg_functions).reset_index()
    
    # Edges - delete connecting edge, put edges to delte node to concat node and aggregate edges connecting same nodes
    edges = edges.loc[~ (((edges['From Number'] == delNode) & (edges['To Number'] == concatTo)) | ((edges['From Number'] == concatTo) & (edges['To Number'] == delNode)))]
    edges.loc[edges['From Number'] == delNode, 'From Number'] = concatTo
    edges.loc[edges['To Number'] == delNode, 'To Number'] = concatTo 
    #Aggregate edges
    agg_functions = {'X': 'mean', 'Lim MVA A': 'sum'}
    edges = edges.groupby(['datetime_beginning_utc','To Number', 'From Number']).aggregate(agg_functions).reset_index()
    
    # Substract one year
    busses['datetime_beginning_utc'] -= pd.DateOffset(years=1)
    edges['datetime_beginning_utc'] -= pd.DateOffset(years=1)
    return busses, edges
    

In [271]:
nodeIndex = busses['bus_id'].max()
bussesConcat, edgesConcat = concat_node(bussesConcat, edgesConcat, nodeIndex)
edgesConcat = edgesConcat.reset_index()

In [276]:
bussesConcat.to_csv("Busses_Timeseries_Table_Set2020.csv")
edgesConcat.to_csv("Edges_Timeseries_Table_Set2020.csv")